In [9]:

from urllib.parse import quote


query = "(AllField:(VR) OR AllField:(Virtual reality) OR AllField:(augmented reality) OR AllField:(AR) OR AllField:(Mixed reality) or OR AllField:(XR)) AND (AllField:(Multiuser) OR AllField:(multi-user) OR AllField:(collaborative))"

def encode_boolean_expression(expression: str) -> str:
    expression = expression.replace(" ", "+")  
    return quote(expression, safe="+") 

query_encoded = encode_boolean_expression(query)
print(query_encoded)


%28AllField%3A%28VR%29+OR+AllField%3A%28Virtual+reality%29+OR+AllField%3A%28augmented+reality%29+OR+AllField%3A%28AR%29+OR+AllField%3A%28Mixed+reality%29+or+OR+AllField%3A%28XR%29%29+AND+%28AllField%3A%28Multiuser%29+OR+AllField%3A%28multi-user%29+OR+AllField%3A%28collaborative%29%29


In [10]:


import requests
from bs4 import BeautifulSoup

# URL for the search
url_base = "https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl"

def add_parameter_to_url(url, parameter_name, parameter_value):
    parameter_connector = "&"
    parameter_assignment_str = "="
    return url + parameter_connector + parameter_name + parameter_assignment_str + parameter_value

query_name = "AllField"

url_base = add_parameter_to_url(url_base, query_name, query_encoded)

page_prm_str = "pageSize"
page_size = "2"
url_base = add_parameter_to_url(url_base, page_prm_str, page_size) 

page_count_str = "startPage"
page_count = 1
url_page = add_parameter_to_url(url_base, page_count_str, str(page_count))



In [11]:

import requests
from bs4 import BeautifulSoup
import time

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"
}


def get_total_query_results(soup: BeautifulSoup) -> int:
    result_count_tag = soup.find(class_="result__count") 
    result_count = int(result_count_tag.get_text().replace(" Results", "").replace(",",""))
    return result_count



In [12]:


def get_acm_papers_page(soup: BeautifulSoup):
    for item in soup.select("li.search__item"):
        publication_type = item.find("div.issue-heading")
        date = item.find("div.bookPubDate ")
        date_str = date['data-title'].replace('Published: ','').strip()
        date_lst = date_str.split(' ')
        date_day = date_lst[0]
        date_month = date_lst[1]
        date_year = date_lst[2]
        title_info = item.find(".issue-item__title a")
        title = title_info.get_text(strip=True)
        link = title_info['href']
        print(date_str + ' - ' + title + ' - ' + link)

def get_acm_papers(url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Failed to retrieve page")
        return []
    
    soup = BeautifulSoup(response.content, "html.parser")
    result_count = get_total_query_results(soup)
    print('Result count: ' + str(result_count))
    get_acm_papers_page(soup)

get_acm_papers(url_page)

Result count: 173565


TypeError: 'NoneType' object is not subscriptable